# CODE TO EXPLORE AND CLUSTER NEIGHBORHOOD IN TORONTO

## Importing relevant libraries

In [6]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


Libraries imported.


## Loading geospatial coordinates from csv file

In [7]:
geodata = pd.read_csv("Geospatial_Coordinates.csv") 


In [8]:
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
geodata.columns = ['PostalCode','Latitude','Longitude']

In [10]:
geodata.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Loading Toronto neighborhood csv file created from previous assignment

In [11]:
df_toronto=pd.read_csv("df_toronto.csv")

In [12]:
df_toronto.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood\n, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae\n


## Joining the Two Dataframes geodata and df_toronto

In [32]:
geodata_neighborhood =df_toronto.join(geodata.set_index('PostalCode'), on='PostalCode')

geodata_neighborhood.head(16)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood\n, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae\n,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park\n, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West\n",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West\n",43.692657,-79.264848


##  With the coordinates of each postal code, a map of Toronto with markers indicating the Postcode position is generated

In [34]:
map = folium.Map(location=[43.6532,-79.3832], zoom_start=11)

for location in geodata_neighborhood.itertuples(): #iterate each row of the dataframe
    label = 'Postal Code: {};  Borough: {};  Neighborhoods: {}'.format(location[1], location[2], location[3])
    label = folium.Popup(label, parse_html=True)    
    folium.CircleMarker(
        [location[-2], location[-1]],
        radius=1,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map) 
    folium.Circle(
        radius=500,
        popup=label,
        location=[location[-2], location[-1]],
        color='#3186cc',
        fill=True,
        fill_color='#3186cc'
    ).add_to(map) 
    
map

## Use geopy library to get the latitude and longitude values of Toronto.

### In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent toronto_explorer, as shown below.

In [35]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, ON, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, ON, Canada are 43.653963, -79.387207.


## Create a map of Toronto with neighborhoods superimposed on top.

In [36]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough,neighborhood in zip(geodata_neighborhood['Latitude'],geodata_neighborhood['Longitude'], geodata_neighborhood['Borough'], geodata_neighborhood['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

The map shows that the Postal codes are not evenly spaced, and the area cover by some of them, using a radius of 500 meters, overlaps. A different radius for each postcode results in a better venues search because that will avoid misrepresentation of the number of venues per postcode caused by too large or low radius values.

folium is a great visualization library. Feel free to zoom into the above map, we can  click on each circle mark to reveal the name of the neighborhood and its respective borough.

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Manhattan data.



In [39]:
Scarborough_data = geodata_neighborhood[geodata_neighborhood['Borough'] == 'Scarborough'].reset_index(drop=True)
Scarborough_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood\n, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae\n,43.773136,-79.239476


Let us proceed and get the corodinate of Scarborough

In [42]:
address = 'Scarborough, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough, ON, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough, ON, Canada are 43.773077, -79.257774.


As we did for Toronto city, let's visualize the neighborhoods Scarborough .




In [44]:
# create map of New York using latitude and longitude values
map_Scarborough = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(Scarborough_data['Latitude'], Scarborough_data['Longitude'], Scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Scarborough)  
    
map_Scarborough

Utilizing the Foursquare API to explore the neighborhoods and segment them

## Define Foursquare Credentials and Version


In [46]:
CLIENT_ID = 'UMHYJ0MJCMYPB2RMI0PUJKHGW2HH32CY0KLM4WXCSSFBIPDB' # My Foursquare ID
CLIENT_SECRET = '2G23RBVRACRMLOXE1CTZ1P5D24DAAZWOZYWZPEG42K3FJ0I2' # My Foursquare Secret
VERSION = '20190818' # Foursquare API version
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UMHYJ0MJCMYPB2RMI0PUJKHGW2HH32CY0KLM4WXCSSFBIPDB
CLIENT_SECRET:2G23RBVRACRMLOXE1CTZ1P5D24DAAZWOZYWZPEG42K3FJ0I2



## Exploring the first neighborhood of Scarborough

In [47]:
# Name of the neighborhood
first_nei=Scarborough_data['Neighborhood'][0]
first_nei

'Rouge, Malvern'

In [48]:
#Location of the neighborhood
first_nei_lat=Scarborough_data.loc[0,'Latitude']
first_nei_lon=Scarborough_data.loc[0,'Longitude']
print('Latitude and longitude values of {} are {}, {}.'.format(first_nei, 
                                                               first_nei_lat, 
                                                               first_nei_lon))

Latitude and longitude values of Rouge, Malvern are 43.806686299999996, -79.19435340000001.


### Now, let's get the top 100 venues that are in Malvern within a radius of 500 meters.



In [51]:
radius = 500 
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    first_nei_lat, 
    first_nei_lon, 
    radius, 
    LIMIT)

In [52]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d59558c00bad70025983509'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

Exploring another neighborhood Since we havent received much results in this current task


In [53]:
second_nei_name=Scarborough_data.loc[15,'Neighborhood']
second_nei_lat=Scarborough_data.loc[15,'Latitude']
second_nei_lon=Scarborough_data.loc[15,'Longitude']
print('Latitude and longitude values of {} are {}, {}.'.format(second_nei_name, 
                                                               second_nei_lat, 
                                                               second_nei_lon))

Latitude and longitude values of L'Amoreaux West
 are 43.799525200000005, -79.3183887.


In [54]:
radius = 500 
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    second_nei_lat, 
    second_nei_lon, 
    radius, 
    LIMIT)

In [55]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d595639787dba0033f8a3bd'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': "L'Amoreaux",
  'headerFullLocation': "L'Amoreaux, Toronto",
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 12,
  'suggestedBounds': {'ne': {'lat': 43.80402520450001,
    'lng': -79.31216562672927},
   'sw': {'lat': 43.7950251955, 'lng': -79.32461177327073}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4df28ea0e4cda09e6da0129a',
       'name': 'Mr Congee Chinese Cuisine 龍粥記',
       'location': {'address': '2900 Warden Ave',
        'crossStreet': 'at Finch Ave. E',
        'lat': 43.798878792587615,
        'lng': -79.3183345011537,
        'labeled

In [56]:
results['response']['groups'][0]['items'][0]['venue']['categories'][0]['name']

'Chinese Restaurant'

The information is in the items key. Lets create a get_category_type function .


In [57]:
venues=results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
nearby_venues.columns

Index(['reasons.count', 'reasons.items', 'referralId', 'venue.categories',
       'venue.id', 'venue.location.address', 'venue.location.cc',
       'venue.location.city', 'venue.location.country',
       'venue.location.crossStreet', 'venue.location.distance',
       'venue.location.formattedAddress', 'venue.location.labeledLatLngs',
       'venue.location.lat', 'venue.location.lng',
       'venue.location.neighborhood', 'venue.location.postalCode',
       'venue.location.state', 'venue.name', 'venue.photos.count',
       'venue.photos.groups'],
      dtype='object')

In [59]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
                return categories_list[0]['name']


In [60]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues


,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Mr Congee Chinese Cuisine 龍粥記,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",43.798879,-79.318335
1,Phoenix Restaurant 金鳳餐廳,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",43.798198,-79.318432
2,Subway,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",43.798983,-79.318838
3,Price Chopper,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",43.799445,-79.318563
4,KFC,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",43.798938,-79.318854
5,Shoppers Drug Mart,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",43.799966,-79.317985
6,Tim Hortons,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",43.799102,-79.318715
7,McDonald's,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",43.798880,-79.318724
8,Pizza Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",43.797909,-79.318113
9,Rogers Plus,"[{'id': '4bf58dd8d48988d122951735', 'name': 'E...",43.798911,-79.318277


In [61]:
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Mr Congee Chinese Cuisine 龍粥記,Chinese Restaurant,43.798879,-79.318335
1,Phoenix Restaurant 金鳳餐廳,Chinese Restaurant,43.798198,-79.318432
2,Subway,Sandwich Place,43.798983,-79.318838
3,Price Chopper,Grocery Store,43.799445,-79.318563
4,KFC,Fast Food Restaurant,43.798938,-79.318854
5,Shoppers Drug Mart,Pharmacy,43.799966,-79.317985
6,Tim Hortons,Coffee Shop,43.799102,-79.318715
7,McDonald's,Fast Food Restaurant,43.798880,-79.318724
8,Pizza Pizza,Pizza Place,43.797909,-79.318113
9,Rogers Plus,Electronics Store,43.798911,-79.318277


# #  Digging deep to Explore more Neighborhoods in Scarborough

###  Let's create a function to repeat the same process to all the neighborhoods in Scarborough

In [62]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Following script run the above function on each neighborhood and create a new dataframe called scarborough_venues.



In [64]:
scarborough_venues = getNearbyVenues(names=Scarborough_data['Neighborhood'],
                                   latitudes=Scarborough_data['Latitude'],
                                   longitudes=Scarborough_data['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood
, Morningside, West Hill
Woburn
Cedarbrae

Scarborough Village
East Birchmount Park
, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West

Birch Cliff, Cliffside West

Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners
, Sullivan
, Tam O'Shanter
Agincourt North, L'Amoreaux East
, Milliken, Steeles East

L'Amoreaux West

Upper Rouge


In [65]:
print(scarborough_venues.shape)
scarborough_venues.head()

(85, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Guildwood\n, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood\n, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


### Venues per neighborhood


In [67]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East\n, Milliken, Steeles East\n",2,2,2,2,2,2
"Birch Cliff, Cliffside West\n",4,4,4,4,4,4
Cedarbrae\n,7,7,7,7,7,7
"Clairlea, Golden Mile, Oakridge",10,10,10,10,10,10
"Clarks Corners\n, Sullivan\n, Tam O'Shanter",11,11,11,11,11,11
"Cliffcrest, Cliffside, Scarborough Village West\n",2,2,2,2,2,2
"Dorset Park, Scarborough Town Centre, Wexford Heights",7,7,7,7,7,7
"East Birchmount Park\n, Ionview, Kennedy Park",5,5,5,5,5,5


### Unique Categories

In [71]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 49 uniques categories.


## Analyze Each Neighborhood


In [72]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]
scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Grocery Store,Hakka Restaurant,History Museum,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood\n, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,"Guildwood\n, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [73]:
scarborough_onehot.shape   #90 venues and 56 unique categories

(85, 50)

### let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category¶


In [74]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Grocery Store,Hakka Restaurant,History Museum,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.250000,0.0,0.0,0.00,0.000000,0.000000,0.25,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.25,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.25,0.0,0.000000,0.000000
1,"Agincourt North, L'Amoreaux East\n, Milliken, ...",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.5,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000
2,"Birch Cliff, Cliffside West\n",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.25,0.000000,0.000000,0.00,0.000000,0.25,0.0,0.0,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.25,0.0,0.000000,0.000000
3,Cedarbrae\n,0.0,0.142857,0.000000,0.142857,0.142857,0.0,0.000000,0.0,0.0,0.00,0.142857,0.000000,0.00,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.142857,0.00,0.000000,0.142857,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.142857,0.000000
4,"Clairlea, Golden Mile, Oakridge",0.0,0.000000,0.000000,0.200000,0.000000,0.0,0.000000,0.2,0.1,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.0,0.000000,0.100000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.100000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.1,0.000000,0.000000,0.0,0.000000,0.1,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.1,0.000000,0.000000
5,"Clarks Corners\n, Sullivan\n, Tam O'Shanter",0.0,0.000000,0.000000,0.000000,0.090909,0.0,0.090909,0.0,0.0,0.00,0.000000,0.090909,0.00,0.000000,0.00,0.0,0.0,0.000000,0.090909,0.090909,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.090909,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.090909,0.0,0.000000,0.090909,0.181818,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.090909,0.000000
6,"Cliffcrest, Cliffside, Scarborough Village West\n",0.5,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.5,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000
7,"Dorset Park, Scarborough Town Centre, Wexford ...",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000000,0.142857,0.00,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.285714,0.000000,0.000000,0.00,0.142857,0.142857,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.142857,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.142857
8,"East Birchmount Park\n, Ionview, Kennedy Park",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.2,0.00,0.000000,0.200000

In [75]:
scarborough_grouped.shape

(16, 50)

### Each neighborhood along with the top 5 most common venues


In [77]:
num_top_venues = 5
for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp =scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
            venue  freq
0  Clothing Store  0.25
1    Skating Rink  0.25
2          Lounge  0.25
3  Breakfast Spot  0.25
4            Park  0.00


----Agincourt North, L'Amoreaux East
, Milliken, Steeles East
----
                       venue  freq
0                       Park   0.5
1                 Playground   0.5
2               Noodle House   0.0
3          Korean Restaurant   0.0
4  Latin American Restaurant   0.0


----Birch Cliff, Cliffside West
----
                   venue  freq
0        College Stadium  0.25
1           Skating Rink  0.25
2  General Entertainment  0.25
3                   Café  0.25
4    American Restaurant  0.00


----Cedarbrae
----
                venue  freq
0     Thai Restaurant  0.14
1              Bakery  0.14
2                Bank  0.14
3  Athletics & Sports  0.14
4    Hakka Restaurant  0.14


----Clairlea, Golden Mile, Oakridge----
          venue  freq
0        Bakery   0.2
1      Bus Line   0.2
2  Intersection   0.1
3  Soccer Field

###  Function to sort the venues in descending order.



In [78]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [79]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Skating Rink,Breakfast Spot,Lounge,Vietnamese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
1,"Agincourt North, L'Amoreaux East\n, Milliken, ...",Playground,Park,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,"Birch Cliff, Cliffside West\n",Skating Rink,General Entertainment,College Stadium,Café,Vietnamese Restaurant,Clothing Store,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
3,Cedarbrae\n,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,College Stadium,Grocery Store,General Entertainment
4,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Metro Station,Soccer Field,Intersection,Fast Food Restaurant,Bus Station,Park,Vietnamese Restaurant,Fried Chicken Joint
5,"Clarks Corners\n, Sullivan\n, Tam O'Shanter",Pizza Place,Fried Chicken Joint,Thai Restaurant,Fast Food Restaurant,Bank,Italian Restaurant,Breakfast Spot,Pharmacy,Noodle House,Chinese Restaurant
6,"Cliffcrest, Cliffside, Scarborough Village West\n",American Restaurant,Motel,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,College Stadium
7,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Pet Store,Chinese Restaurant,Latin American Restaurant,Light Rail Station,Vietnamese Restaurant,Rental Car Location,Caribbean Restaurant,Fast Food Restaurant,Electronics Store
8,"East Birchmount Park\n, Ionview, Kennedy Park",Chinese Restaurant,Discount Store,Department Store,Bus Station,Coffee Shop,Clothing Store,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
9,"Guildwood\n, Morningside, West Hill",Mexican Restaurant,Intersection,Electronics Store,Rental Car Location,Pizza Place,Breakfast Spot,Medical Center,Vietnamese Restaurant,Fried Chicken Joint,Fast Food Restaurant


### Cluster Neighborhoods

In [80]:
# set number of clusters
# Using k-means to cluster the neighborhood into 4 clusters.
kclusters = 4

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 3, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 2, 3, 0])

### Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

#### Since the neighborhood 'Upper Rouge' has no venue data returned lets remove that neighborhood

In [81]:
scarborough_merged = Scarborough_data[0:16]

# add clustering labels
scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarborough_merged.head() # check the last columns!

C:\Users\Akatamba\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1,Fast Food Restaurant,Vietnamese Restaurant,History Museum,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,College Stadium
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,3,Bar,History Museum,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,M1E,Scarborough,"Guildwood\n, Morningside, West Hill",43.763573,-79.188711,1,Mexican Restaurant,Intersection,Electronics Store,Rental Car Location,Pizza Place,Breakfast Spot,Medical Center,Vietnamese Restaurant,Fried Chicken Joint,Fast Food Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Indian Restaurant,Korean Restaurant,Clothing Store,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,M1H,Scarborough,Cedarbrae\n,43.773136,-79.239476,1,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,College Stadium,Grocery Store,General Entertainment


## Visualising the clusters

In [84]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)
# add markers to the map
markers_colors = []
for lat, lon, nei , cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#2adddd', '#d4dd80', '#ff0000']


## Examine Clusters

### Examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster.

##### Cluster 1

In [85]:

scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,"Clairlea, Golden Mile, Oakridge",0,Bakery,Bus Line,Metro Station,Soccer Field,Intersection,Fast Food Restaurant,Bus Station,Park,Vietnamese Restaurant,Fried Chicken Joint
8,"Cliffcrest, Cliffside, Scarborough Village West\n",0,American Restaurant,Motel,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,College Stadium
11,"Maryvale, Wexford",0,Middle Eastern Restaurant,Auto Garage,Shopping Mall,Sandwich Place,Breakfast Spot,Vietnamese Restaurant,Clothing Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
15,L'Amoreaux West\n,0,Chinese Restaurant,Fast Food Restaurant,Coffee Shop,Grocery Store,Sandwich Place,Electronics Store,Pizza Place,Pharmacy,Breakfast Spot,General Entertainment


#### Cluster 2

In [87]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge, Malvern",1,Fast Food Restaurant,Vietnamese Restaurant,History Museum,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,College Stadium
2,"Guildwood\n, Morningside, West Hill",1,Mexican Restaurant,Intersection,Electronics Store,Rental Car Location,Pizza Place,Breakfast Spot,Medical Center,Vietnamese Restaurant,Fried Chicken Joint,Fast Food Restaurant
3,Woburn,1,Coffee Shop,Indian Restaurant,Korean Restaurant,Clothing Store,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,Cedarbrae\n,1,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,College Stadium,Grocery Store,General Entertainment
5,Scarborough Village,1,Playground,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
6,"East Birchmount Park\n, Ionview, Kennedy Park",1,Chinese Restaurant,Discount Store,Department Store,Bus Station,Coffee Shop,Clothing Store,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
9,"Birch Cliff, Cliffside West\n",1,Skating Rink,General Entertainment,College Stadium,Café,Vietnamese Restaurant,Clothing Store,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
10,"Dorset Park, Scarborough Town Centre, Wexford ...",1,Indian Restaurant,Pet Store,Chinese Restaurant,Latin American Restaurant,Light Rail Station,Vietnamese Restaurant,Rental Car Location,Caribbean Restaurant,Fast Food Restaurant,Electronics Store
12,Agincourt,1,Clothing Store,Skating Rink,Breakfast Spot,Lounge,Vietnamese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


#### Cluster 3

In [89]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,"Clarks Corners\n, Sullivan\n, Tam O'Shanter",2,Pizza Place,Fried Chicken Joint,Thai Restaurant,Fast Food Restaurant,Bank,Italian Restaurant,Breakfast Spot,Pharmacy,Noodle House,Chinese Restaurant


#### Cluster 4

In [90]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Highland Creek, Rouge Hill, Port Union",3,Bar,History Museum,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
14,"Agincourt North, L'Amoreaux East\n, Milliken, ...",3,Playground,Park,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store


#### Cluster 5

In [91]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


##                                      Good Bye